### Conexion con aws

In [52]:
import boto3

# Crear cliente de DynamoDB
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')

# Listar tablas
try:
    response = dynamodb_client.list_tables()
    tablas = response.get('TableNames', [])
    print("Tablas en DynamoDB:")
    for tabla in tablas:
        print(f"- {tabla}")
except Exception as e:
    print("Error al listar tablas:", str(e))

Tablas en DynamoDB:


### 1 -Crear al menos 3 tablas con dos atributos cada una

In [53]:
# Crear tablas en DynamoDB
try:
    # Tabla 1
    dynamodb_client.create_table(
        TableName='Tabla1',
        KeySchema=[
            {'AttributeName': 'Nombre', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'Apellido', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'Nombre', 'AttributeType': 'S'},
            {'AttributeName': 'Apellido', 'AttributeType': 'S'}
            
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

    # Tabla 2
    dynamodb_client.create_table(
        TableName='Tabla2',
        KeySchema=[
            {'AttributeName': 'Nombre', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'Formacion', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'Nombre', 'AttributeType': 'S'},
            {'AttributeName': 'Formacion', 'AttributeType': 'S'}
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

    # Tabla 3
    dynamodb_client.create_table(
        TableName='Tabla3',
        KeySchema=[
            {'AttributeName': 'Nombre', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'Profesion', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'Nombre', 'AttributeType': 'S'},
            {'AttributeName': 'Profesion', 'AttributeType': 'S'}

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

    print("Tablas creadas exitosamente.")
except Exception as e:
    print("Error al crear tablas:", str(e))

Tablas creadas exitosamente.


### 2 - Crear tres registros

In [55]:
# Crear registros en las tablas
try:
    # Registro en Tabla1
    dynamodb_client.put_item(
        TableName='Tabla1',
        Item={
            'Nombre': {'S': 'Juan'},
            'Apellido': {'S': 'Perez'},
            'Edad': {'N': '17'}

        }
    )

    # Registro en Tabla2
    dynamodb_client.put_item(
        TableName='Tabla2',
        Item={
            'Nombre': {'S': 'Ana'},
            'Formacion': {'S': 'Ingenieria'},
            'Edad': {'N': '28'}
        }
    )

    # Registro en Tabla3
    dynamodb_client.put_item(
        TableName='Tabla3',
        Item={
            'Nombre': {'S': 'Luis'},
            'Profesion': {'S': 'Medico'},
            'Edad': {'N': '48'}
        }
    )

    print("Registros creados exitosamente.")
except Exception as e:
    print("Error al crear registros:", str(e))

Registros creados exitosamente.


### 3 - Obtener un registro de cada tabla

In [56]:
def obtener_un_registro_por_tabla():
    """
    Esta función se conecta a AWS DynamoDB, lista todas las tablas disponibles
    y recupera un único registro de cada tabla (si existe).

    Retorna:
        - Un diccionario donde las claves son los nombres de las tablas 
          y los valores son los primeros registros encontrados.
    """
    # Obtener lista de tablas
    response = dynamodb_client.list_tables()
    tablas = response.get('TableNames', [])

    resultados = {}

    for tabla in tablas:
        # Obtener un único registro de la tabla
        response = dynamodb_client.scan(TableName=tabla, Limit=1)

        # Extraer el primer ítem si existe
        items = response.get('Items', [])
        if items:
            resultados[tabla] = items[0]
        else:
            resultados[tabla] = None  # Si la tabla está vacía

    return resultados

# Ejecutar la función y mostrar los resultados
if __name__ == "__main__":
    registros = obtener_un_registro_por_tabla()
    for tabla, registro in registros.items():
        print(f"Tabla: {tabla}, Registro: {registro}")

Tabla: Tabla1, Registro: {'Nombre': {'S': 'Juan'}, 'Edad': {'N': '17'}, 'Apellido': {'S': 'Perez'}}
Tabla: Tabla2, Registro: {'Nombre': {'S': 'Ana'}, 'Edad': {'N': '28'}, 'Formacion': {'S': 'Ingenieria'}}
Tabla: Tabla3, Registro: {'Nombre': {'S': 'Luis'}, 'Edad': {'N': '48'}, 'Profesion': {'S': 'Medico'}}


#### 4 - Actualizar un registro de cada tabla

In [61]:
# Crear el cliente de DynamoDB
dynamodb = boto3.resource('dynamodb')

# Función para actualizar el registro de cada tabla
def actualizar_edad(tabla_nombre, nombre_usuario, clave_ordenacion, nueva_edad):
    try:
        # Obtener la referencia de la tabla
        tabla = dynamodb.Table(tabla_nombre)

        # Ejecutar la operación de actualización
        response = tabla.update_item(
            Key={
                'Nombre': nombre_usuario,  # Clave de partición
                'Apellido': clave_ordenacion if tabla_nombre == 'Tabla1' else 
                'Formacion': clave_ordenacion if tabla_nombre == 'Tabla2' else
                'Profesion': clave_ordenacion  # Clave de ordenación
            },
            UpdateExpression="set Edad = :e",  # Actualizamos el atributo Edad
            ExpressionAttributeValues={
                ':e': nueva_edad  # El nuevo valor para Edad
            },
            ReturnValues="ALL_NEW"  # Devuelve los valores después de la actualización
        )
        
        # Mostrar los resultados de la actualización
        print(f"Actualización en {tabla_nombre}: {response['Attributes']}")
    except Exception as e:
        print(f"Error al actualizar en {tabla_nombre}: {e}")
# Actualizar los registros en las tablas
if __name__ == "__main__":
    # Actualizar la Edad de los registros
    actualizar_edad('Tabla1', 'Juan', 23)  # Cambiar Edad de Juan en Tabla1 a 23
    actualizar_edad('Tabla2', 'Ana', 33)   # Cambiar Edad de Ana en Tabla2 a 33
    actualizar_edad('Tabla3', 'Luis', 64)  # Cambiar Edad de Luis en Tabla3 a 64



SyntaxError: invalid syntax (3286356775.py, line 15)